# Linear Regression Dask Test

Testing linear regression using Dask

### Setup

In [1]:
# Extension reloader to import a function again when re-running cell 
%load_ext autoreload
%autoreload 2

### Load Configuration

In [2]:
"""
Loads common configuration parameters
"""
import utils.configuration_manager as configuration_manager
from pathlib import PurePath
from os import getcwd

config_path = PurePath(getcwd(),'config.ini')
config = configuration_manager.Config(config_path)

# Assumes parquet directory as input
input_path = config.input_path
print('Input path: '+ input_path)

# For result storage
output_directory = config.output_directory
print('Output path: ' + output_directory)

Loading configuration from: /home/justin/Code/interpretability_experiment/config.ini
Input path: data/2018_Yellow_Taxi_Trip_Data_float64
Output path: output


### Start local Dask Client

In [3]:
from dask.distributed import Client, LocalCluster
try:
    if client:
        print('Restarting client')
        client.restart()
except:
#     cluster = LocalCluster(dashboard_address=':20100', memory_limit='4G')
    cluster = LocalCluster(dashboard_address=':20100')
    print('Setting new client')
    client = Client(cluster)
    print(client)
client

Setting new client
<Client: 'tcp://127.0.0.1:44641' processes=5 threads=10, memory=16.39 GB>


Client Scheduler: tcp://127.0.0.1:44641 Dashboard: http://127.0.0.1:20100/status,Cluster Workers: 5 Cores: 10 Memory: 16.39 GB


### Dask dataframe loader

In [5]:
import dask.dataframe as dd
import fastparquet

In [6]:
ddf = dd.read_parquet(input_path)

In [7]:
ddf.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
index,,,,,,,,,,,,,,,,,
0,1,2018-12-03 09:58:01,2018-12-03 10:14:17,1.0,1.20,1,N,186,161,1,11.0,0.0,0.5,2.95,0.00,0.3,14.75
1,2,2018-12-03 09:41:32,2018-12-03 10:20:08,1.0,12.03,1,N,138,162,1,39.0,0.0,0.5,9.11,5.76,0.3,54.67
2,2,2018-12-03 08:54:36,2018-12-03 08:59:35,2.0,0.86,1,N,151,166,1,5.5,0.0,0.5,1.26,0.00,0.3,7.56
3,2,2018-12-03 09:02:08,2018-12-03 09:07:16,2.0,1.09,1,N,166,238,1,6.0,0.0,0.5,1.36,0.00,0.3,8.16
4,2,2018-12-03 09:10:10,2018-12-03 09:21:32,2.0,1.78,1,N,238,75,1,9.5,0.0,0.5,2.06,0.00,0.3,12.36


Ignoring categorical data for simplicity

### Define what we are trying to model

In [8]:
target = 'tip_amount'

In [9]:
"""
Subtract the tip_amount from the total_amount to prevent any leakage, 
using a new total_amount_wo_tip column.
"""
ddf['total_amount_wo_tip'] = ddf['total_amount'] - ddf['tip_amount']

In [10]:
# Select all numerical columns as inputs
input_columns = ddf.select_dtypes(['float']).columns

In [11]:
# Remove unwanted numerical columns 
input_columns = [col for col in input_columns if col not in [target, 'total_amount']]

In [12]:
print(input_columns)

['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'total_amount_wo_tip']


In [13]:
ddf[input_columns].head()

,passenger_count,trip_distance,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,total_amount_wo_tip
index,,,,,,,,
0,1.0,1.20,11.0,0.0,0.5,0.00,0.3,11.80
1,1.0,12.03,39.0,0.0,0.5,5.76,0.3,45.56
2,2.0,0.86,5.5,0.0,0.5,0.00,0.3,6.30
3,2.0,1.09,6.0,0.0,0.5,0.00,0.3,6.80
4,2.0,1.78,9.5,0.0,0.5,0.00,0.3,10.30


In [14]:
ddf[target].head()

index
0    2.95
1    9.11
2    1.26
3    1.36
4    2.06
Name: tip_amount, dtype: float64

### Preparing dataset for dask

In [16]:
"""
Get the lengths of each block to allow conversion to DF
https://nbviewer.jupyter.org/github/PuneetGrov3r/MediumPosts/blob/master/Tackle/BigData-IncrementalLearningAndDask.ipynb#Method-2:-Using-Dask:
"""
lengths = []
for part in ddf.partitions:
    l = part.shape[0].compute()
    lengths.append(l)
#     print(l, part.shape[1])

In [17]:
# Set X, y to load as dask arrays
X, y = ddf[input_columns].to_dask_array(lengths=lengths) , ddf[target].to_dask_array(lengths=lengths)

In [18]:
"""
Resizing blocks in order to prevent broadcasting errors due to different input sizes
"""
chunk_length = 200000
import dask
from dask_ml.preprocessing import RobustScaler

Xo = dask.array.zeros((X.shape[0],1), chunks=(chunk_length,1))

for i, col_ in enumerate(ddf[input_columns + [target]].columns):
    if col_ == target:
        rsc = RobustScaler()
        y = rsc.fit_transform(y.reshape(-1, 1)).reshape(1, -1)[0]
    else:
        rsc = RobustScaler()
        temp = rsc.fit_transform(X[:,i].reshape(-1, 1))
        Xo = dask.array.concatenate([Xo, temp], axis=1)

In [19]:
Xo = Xo[:, 1:]

In [20]:
# Check
Xo[-5:].compute()

array([[ 0.        , -0.48412698, -0.52380952, -0.5       ,  0.        ,
         0.        ,  0.        , -0.55      ],
       [ 0.        ,  0.08333333, -0.0952381 , -0.5       ,  0.        ,
         0.        ,  0.        , -0.1       ],
       [ 0.        ,  0.03968254,  0.        , -0.5       ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        , -0.29761905, -0.33333333, -0.5       ,  0.        ,
         0.        ,  0.        , -0.35      ],
       [ 0.        ,  0.35714286,  0.47619048, -0.5       ,  0.        ,
         0.        ,  0.        ,  0.5       ]])

In [21]:
Xo = Xo.rechunk({1: Xo.shape[1]})
Xo = Xo.rechunk({0: chunk_length})
y = y.rechunk({0: chunk_length})

### Train/validation/test prep

In [22]:
tr_len = int(0.8*Xo.shape[0])
print(tr_len)

89787700


In [23]:
xtrain, ytrain = Xo[:tr_len], y[:tr_len]
xvalid, yvalid = Xo[tr_len:], y[tr_len:]
xtrain.shape, ytrain.shape, xvalid.shape, yvalid.shape

((89787700, 8), (89787700,), (22446926, 8), (22446926,))

### Train LR model

In [24]:
from dask_ml.linear_model import LinearRegression

In [25]:
est = LinearRegression()

In [26]:
%time est.fit(xtrain, y=ytrain)

CPU times: user 1h 1min 19s, sys: 4min 45s, total: 1h 6min 5s
Wall time: 3h 16min 5s


LinearRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                 intercept_scaling=1.0, max_iter=100, multi_class='ovr',
                 n_jobs=1, penalty='l2', random_state=None, solver='admm',
                 solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False)

In [27]:
preds = est.predict(xvalid)

In [28]:
%time preds[0:10].compute()

CPU times: user 77.6 ms, sys: 44 µs, total: 77.7 ms
Wall time: 272 ms


array([ 0.15359374,  0.3479344 , -0.03934699,  0.08588969,  1.10204229,
       -0.20595782, -0.27886385, -0.32213797, -0.02296863, -0.04164265])

In [74]:
# import matplotlib.pyplot as plt

In [75]:
# plt.scatter(preds.compute(), yvalid.compute())

### Test Model

In [38]:
preds.shape

(22446926,)

In [39]:
yvalid.shape

(22446926,)

In [46]:
# MAE
%time (abs(preds-yvalid)).mean(axis=0).compute()

CPU times: user 4.73 s, sys: 245 ms, total: 4.98 s
Wall time: 6.89 s


0.48113266567598045

In [45]:
# MSE
%time ((preds-yvalid)**2).mean(axis=0).compute()

CPU times: user 4.66 s, sys: 338 ms, total: 5 s
Wall time: 9.2 s


7.723888769609159

Although this isn't an apples to apples comparison as only numerical inputs were used in this regression (and not categoricals) and more data was used to train the model, it seems the linear model over the entire dataset offers a slightly lower MAE but higher MSE relative to the LightGBM model.  

This means that prediction are closer on average, but that the mistaken predictions are farther from the mark. 

### Save model

In [47]:
print(type(est))

<class 'dask_ml.linear_model.glm.LinearRegression'>


In [48]:
filename = 'trained_models/lr_estimator.sav'
import joblib

In [49]:
joblib.dump(est, filename)

['trained_models/lr_estimator.sav']

In [50]:
test_est = joblib.load(filename)

In [51]:
# Check model
preds = test_est.predict(xvalid)

In [52]:
# MAE
%time (abs(preds-yvalid)).mean(axis=0).compute()

CPU times: user 4.69 s, sys: 315 ms, total: 5 s
Wall time: 6.41 s


0.48113266567598045

### Evaluate weights

In [57]:
est.coef_

array([-0.00700196,  0.08453412,  0.14894866,  0.02447336, -0.09738848,
        0.06014647, -0.00539934,  0.14216559])

In [58]:
import eli5

/home/justin/.conda/envs/ml_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/justin/.conda/envs/ml_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [61]:
print(type(est))

<class 'dask_ml.linear_model.glm.LinearRegression'>


In [63]:
from sklearn.linear_model import LinearRegression as sklearn_lr
sklearn_est = sklearn_lr()

In [66]:
# Move regression model to sklearn for api support
sklearn_est.coef_ = est.coef_
sklearn_est.intercept_ = est.intercept_

In [72]:
print(input_columns)

['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'total_amount_wo_tip']


In [73]:
eli5.explain_weights(sklearn_est, 
                     feature_names=input_columns, 
                     target_names=target)

Weight?,Feature
+0.149,fare_amount
+0.142,total_amount_wo_tip
+0.085,trip_distance
+0.060,tolls_amount
+0.024,extra
-0.005,improvement_surcharge
-0.007,passenger_count
-0.057,<BIAS>
-0.097,mta_tax
